# Visualize encodings for geospatial entities in a tile.


In [ ]:
import pandas as pd
import numpy as np
import glob
import geopandas
import os
import pickle

from geo_encodings import MPPEncoder

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter
from geo_encodings import draw_shape


## Setup

In [ ]:
# This is the dimension of the (square) AOIs. Set thi to match what was used
# when the tiles were created.
aoi_size = 1000

# This is the resolution of the MPP encoding.
resolution = 50

## Data prep

In [ ]:
# Read the list of labels.
fname = 'data/labels.csv'
labels = pd.read_csv(fname)
label_count = len(labels)
print('%d labels' % label_count)

label_id_lookup = {
    z['label']: z['id']
    for z in labels.to_dict('records')
}

In [ ]:
# Set up the MPP encoder.
encoder = MPPEncoder(
    region=[0, 0, aoi_size, aoi_size],
    resolution=resolution,
    center=True
)
print('%d elements in encodings' % len(encoder))

## Processing

In [ ]:
# Get a list of AOI tags.
globstring = 'data/tiles/*'
tile_dnames = glob.glob(globstring)
aoi_names = [
    z[z.rfind('/') + 1 :]
    for z in tile_dnames
]
print('%d AOIs' % len(aoi_names))


In [ ]:
# Loop over tiles
keepers = []

for aoi_name in aoi_names[:1]:
    print('\nhandling AOI %s' % aoi_name)

    # This will hold all encodings for this AOI
    tile_encodings_for_aoi = []

    # Get a list of all tiles for this AOI.
    globstring = 'data/tiles/%s/*.pq' % aoi_name
    fnames = glob.glob(globstring)
    print('%d tiles' % len(fnames))
    for fname in fnames:
        tile_vectors = []
        gdf = geopandas.read_parquet(fname)
        if len(gdf) < 80:
            continue
        print('%d elements in tile' % len(gdf))
        
        # for _, rec in gdf.iterrows():
        for _, rec in gdf.sample(frac=1).iterrows():
            encoding = encoder.encode(rec['geometry']).values()
            etype = np.zeros(label_count)
            label_string = '%s : %s' % (rec['category'], rec['label'])
            label_id = label_id_lookup[label_string]
            etype[label_id] = 1.0

            # Concatenate the encoding and type vectors.
            vector = np.hstack((etype, encoding))
            tile_vectors.append(vector)
        tile_encoding = np.vstack(tile_vectors)
        tile_encodings_for_aoi.append(tile_encoding)

        if len(gdf) >= 80:
            keepers.append(gdf)

    # # Save all of the encodings for this tile.
    # fname = 'data/encodings/%s.pkl' % aoi_name
    # os.makedirs(os.path.dirname(fname), exist_ok=True)
    # with open(fname, 'wb') as dest:
    #     pickle.dump(tile_encodings_for_aoi, dest)
    # print('wrote %s' % (fname))
        

In [ ]:
print('%d keepers' % len(keepers))
index = 0
gdf = keepers[index]
data = tile_encodings_for_aoi[index]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Plot as heatmap
plt.figure(figsize=(12, 5))
plt.imshow(data, cmap='viridis', origin='upper', aspect='auto')
plt.colorbar(label='Value')
plt.title('Encoding for a tile')
plt.ylabel('entity / row index')
plt.xlabel('encoding element')
plt.show()


In [ ]:
gdf[['category', 'label']].value_counts().sort_index()

In [ ]:
# Roadways
gents = gdf.to_dict('records')
fig = make_subplots(rows=1, cols=1)
shown = set()
shown.add('land')

def add_features(label, color, linewidth):
    for gent in gents:
        if gent['label'] == label:
            draw_shape(gent['geometry'], fig, irow=1, icol=1, 
                       name=label, color=color, linewidth=linewidth,
                       showlegend=(label not in shown))
            shown.add(label)

add_features('land', color='#e0d8b0', linewidth=1)
add_features('highway', color='red', linewidth=3)
add_features('primary road', color='blue', linewidth=2)
add_features('secondary road', color='green', linewidth=2)
add_features('tertiary road', color='teal', linewidth=2)
add_features('residential road', color='gray', linewidth=1)

lo = fig['layout']
lo['title'] = 'roadways'
lo['width'] = 420
lo['height'] = 350
lo['xaxis1']['range'] = [0, 1000]
lo['yaxis1']['range'] = [0, 1000]
lo['xaxis1']['showticklabels'] = False
lo['yaxis1']['showticklabels'] = False
fig

In [ ]:
# Waterways
fig = make_subplots(rows=1, cols=1)
shown = set()
shown.add('land')
shown.add('highway')
shown.add('primary road')
shown.add('secondary road')
shown.add('tertiary teal')
shown.add('residential road')

add_features('land', color='#e0d8b0', linewidth=1)
add_features('highway', color='#bbbbbb', linewidth=1)
add_features('primary road', color='#bbbbbb', linewidth=1)
add_features('secondary road', color='#bbbbbb', linewidth=1)
add_features('tertiary teal', color='#bbbbbb', linewidth=1)
add_features('residential road', color='#bbbbbb', linewidth=1)

add_features('rivers and streams', color='blue', linewidth=2)
add_features('lakes and ponds', color='#71b8f0', linewidth=2)

lo = fig['layout']
lo['title'] = 'waterways'
lo['width'] = 420
lo['height'] = 350
lo['xaxis1']['range'] = [0, 1000]
lo['yaxis1']['range'] = [0, 1000]
lo['xaxis1']['showticklabels'] = False
lo['yaxis1']['showticklabels'] = False
fig

In [ ]:
# Amenities
fig = make_subplots(rows=1, cols=1)
shown = set()
shown.add('land')
shown.add('highway')
shown.add('primary road')
shown.add('secondary road')
shown.add('tertiary teal')
shown.add('residential road')

add_features('land', color='#e0d8b0', linewidth=1)
add_features('highway', color='#bbbbbb', linewidth=1)
add_features('primary road', color='#bbbbbb', linewidth=1)
add_features('secondary road', color='#bbbbbb', linewidth=1)
add_features('tertiary teal', color='#bbbbbb', linewidth=1)
add_features('residential road', color='#bbbbbb', linewidth=1)

add_features('parking lot', color='gray', linewidth=2)
add_features('commercial', color='blue', linewidth=2)
add_features('food and drink', color='purple', linewidth=2)
add_features('recreation', color='green', linewidth=2)

lo = fig['layout']
lo['title'] = 'amenities'
lo['width'] = 420
lo['height'] = 350
lo['xaxis1']['range'] = [0, 1000]
lo['yaxis1']['range'] = [0, 1000]
lo['xaxis1']['showticklabels'] = False
lo['yaxis1']['showticklabels'] = False
fig

In [ ]:
# landuse
fig = make_subplots(rows=1, cols=1)
shown = set()
shown.add('land')
shown.add('highway')
shown.add('primary road')
shown.add('secondary road')
shown.add('tertiary teal')
shown.add('residential road')

add_features('land', color='#e0d8b0', linewidth=1)
add_features('highway', color='#bbbbbb', linewidth=1)
add_features('primary road', color='#bbbbbb', linewidth=1)
add_features('secondary road', color='#bbbbbb', linewidth=1)
add_features('tertiary teal', color='#bbbbbb', linewidth=1)
add_features('residential road', color='#bbbbbb', linewidth=1)

add_features('commercial', color='blue', linewidth=2)
add_features('agricultural', color='green', linewidth=2)
add_features('forest', color='darkgreen', linewidth=2)
add_features('residential', color='magenta', linewidth=2)
add_features('industrial', color='darkgray', linewidth=2)
add_features('retail', color='orange', linewidth=2)

lo = fig['layout']
lo['title'] = 'landuse'
lo['width'] = 420
lo['height'] = 350
lo['xaxis1']['range'] = [0, 1000]
lo['yaxis1']['range'] = [0, 1000]
lo['xaxis1']['showticklabels'] = False
lo['yaxis1']['showticklabels'] = False
fig